# Stackoverflow Q/A : Creating SQL database 

The Stackoverflow data are available at https://archive.org/download/stackexchange. Stackoverflow is a Q/A site where users post questions, answer questions, comment, and vote on posts.

In [1]:
import sqlite3
import xml.etree.ElementTree as ET
import re
from bs4 import BeautifulSoup


source = 'stackoverflowASTRO'
dbfile = '../db/'+source+'.db'
conn = sqlite3.connect(dbfile)
c = conn.cursor()

# Writing POSTS to databases

posts_file = '../xml/stackoverflow/posts.xml'
c.execute('''CREATE TABLE IF NOT EXISTS posts
            (Id INTEGER, PostTypeId INTEGER, ParentId INTEGER, AcceptedAnswerId INTEGER, CreationDate TEXT, Score INTEGER, 
            ViewCount INTEGER, Body TEXT, OwnerUserId INTEGER, LastEditorUserId INTEGER, LastEditorDisplayName TEXT, LastEditDate TEXT,
            LastActivityDate TEXT, CommunityOwnedDate TEXT, ClosedDate TEXT, Title TEXT, Tags TEXT, AnswerCount INTEGER, 
            CommentCount INTEGER, FavoriteCount INTEGER)''')
conn.commit()

columns={"Id":None, 
        "PostTypeId":None, 
        "ParentId":None, 
        "AcceptedAnswerId":None, 
        "CreationDate":None, 
        "Score":None, 
        "ViewCount":None, 
        "Body":None, 
        "OwnerUserId":None, 
        "LastEditorUserId":None, 
        "LastEditorDisplayName":None, 
        "LastEditDate":None, 
        "LastActivityDate":None, 
        "CommunityOwnedDate":None, 
        "ClosedDate":None, 
        "Title":None, 
        "Tags":None, 
        "AnswerCount":None, 
        "CommentCount":None, 
        "FavoriteCount":None}

# Parse XML file
tree = ET.parse(posts_file)
root = tree.getroot()


for child in root:
    name, attrs = (child.tag, child.attrib)
    if name=='row':        
        for (k,v) in columns.iteritems():
            if not attrs.has_key(k):
                attrs[k]=v

        # parse Body using BeautifulSoup        
        if attrs['Body'] is not None:                
            soup = BeautifulSoup(attrs["Body"], "lxml-xml")
            attrs["Body"] = soup.get_text("\n", strip=True).replace("\n",' ')   

        # convert Tags from <a><b> to a,b 
        if attrs['Tags'] is not None: attrs['Tags'] = ','.join(re.findall(r'<([^>]+)', attrs["Tags"]))
            
        t = (attrs["Id"], attrs["PostTypeId"], attrs["ParentId"], attrs["AcceptedAnswerId"], attrs["CreationDate"], 
            attrs["Score"], attrs["ViewCount"], attrs["Body"] , attrs["OwnerUserId"], attrs["LastEditorUserId"], 
            attrs["LastEditorDisplayName"], attrs["LastEditDate"], attrs["LastActivityDate"], attrs["CommunityOwnedDate"], 
            attrs["ClosedDate"], attrs["Title"], attrs['Tags'], attrs["AnswerCount"], attrs["CommentCount"], attrs["FavoriteCount"])
        c.execute("""INSERT INTO posts VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""",t)        
        if int(attrs['Id'])%1000==0:
            conn.commit()


# Writing BADGES to databases

badges_file = '../xml/stackoverflow/badges.xml'
c.execute('''create table IF NOT EXISTS badges
(Id INTEGER, UserId INTEGER, Name TEXT, Date text)''')
conn.commit()

tree = ET.parse(badges_file)
root = tree.getroot()

for child in root:
    name, attrs = (child.tag, child.attrib)
    if name=='row':
        t = (attrs["Id"],attrs["UserId"],attrs["Name"],attrs["Date"])
        c.execute("""INSERT INTO badges VALUES (?,?,?,?)""",t)
        if int(attrs['Id'])%1000==0:
            conn.commit()

            
# Writing COMMENTS to databases

comments_file = '../xml/stackoverflow/comments.xml'
c.execute('''CREATE TABLE IF NOT EXISTS comments (Id INTEGER, PostId INTEGER, Text TEXT, CreationDate TEXT, 
            UserId INTEGER)''')
conn.commit()

columns={"Id":None, 
        "PostId":None, 
        "Text":None, 
        "CreationDate":None, 
        "UserId":None}

tree = ET.parse(comments_file)
root = tree.getroot()

for child in root:
    name, attrs = (child.tag, child.attrib)
    if name=='row':        
        # add default values to attributes if any are missing
        for (k,v) in columns.iteritems():
            if not attrs.has_key(k):
                attrs[k]=v
                
        if attrs['Text'] is not None:                 
            soup = BeautifulSoup(attrs["Text"], 'lxml-xml')
            attrs["Text"] = soup.get_text("\n", strip=True).replace("\n",' ')                
        t = (attrs["Id"],attrs["PostId"],attrs["Text"],attrs["CreationDate"],attrs["UserId"])
        c.execute("""INSERT INTO comments VALUES (?,?,?,?,?)""",t)
        if int(attrs['Id'])%1000==0:
            conn.commit()


# Writing USERS to databases

users_file = '../xml/stackoverflow/users.xml'
c.execute('''create table IF NOT EXISTS users (Id INTEGER, Reputation INTEGER, CreationDate TEXT, DisplayName TEXT, 
            EmailHash TEXT, LastAccessDate TEXT, WebsiteUrl TEXT, Location TEXT, AGE TEXT, AboutMe TEXT, Views INTEGER,
            UpVotes INTEGER, DownVotes INTEGER)''')
conn.commit()

columns={"Id":None,
       "Reputation":None, 
       "CreationDate":None, 
       "DisplayName":None, 
       "EmailHash":None,
       "LastAccessDate":None, 
       "WebsiteUrl":None, 
       "Location":None,
       "Age":None, 
       "AboutMe":None, 
       "Views":None, 
       "UpVotes":None, 
       "DownVotes":None}

# Parse XML file
tree = ET.parse(users_file)
root = tree.getroot()

for child in root:
    name, attrs = (child.tag, child.attrib)
    if name=='row':        
        for (k,v) in columns.iteritems():
            if not attrs.has_key(k):
                attrs[k]=v
                
        t = (attrs["Id"], attrs["Reputation"], attrs["CreationDate"], attrs["DisplayName"], attrs["EmailHash"], 
            attrs["LastAccessDate"], attrs["WebsiteUrl"], attrs["Location"], attrs["Age"], attrs["AboutMe"], 
            attrs["Views"], attrs["UpVotes"], attrs["DownVotes"])
            
        c.execute("""INSERT INTO users VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)""",t)
        if int(attrs['Id'])%1000==0:
            conn.commit()
            

# Writing VOTES to databases

votes_file = '../xml/stackoverflow/votes.xml'
c.execute('''create table IF NOT EXISTS votes (Id INTEGER, PostId INTEGER, VoteTypeId INTEGER, CreationDate TEXT, 
            UserId INTEGER, BountyAmount INTEGER)''')
conn.commit()

columns={"Id":None, 
        "PostId":None, 
        "VoteTypeId":None, 
        "CreationDate":None, 
        "UserId":None, 
        "BountyAmount":None}

# Parse XML file            
tree = ET.parse(votes_file)
root = tree.getroot()

for child in root:
    name, attrs = (child.tag, child.attrib)
    if name=='row':        
        for (k,v) in columns.iteritems():
            if not attrs.has_key(k):
                attrs[k]=v
        t = (attrs["Id"],attrs["PostId"],attrs["VoteTypeId"],attrs["CreationDate"],attrs["UserId"],attrs["BountyAmount"])
        c.execute("""INSERT INTO votes VALUES (?,?,?,?,?,?)""",t)        
        if int(attrs['Id'])%1000==0:
            conn.commit()



In [2]:
dbfile = '../db/stackoverflowASTRO.db'
conn = sqlite3.connect(dbfile)
c = conn.cursor()

In [3]:
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

[(u'posts',), (u'badges',), (u'comments',), (u'users',), (u'votes',)]


In [4]:
c.execute("PRAGMA table_info([posts]);")
c.fetchall()

[(0, u'Id', u'INTEGER', 0, None, 0),
 (1, u'PostTypeId', u'INTEGER', 0, None, 0),
 (2, u'ParentId', u'INTEGER', 0, None, 0),
 (3, u'AcceptedAnswerId', u'INTEGER', 0, None, 0),
 (4, u'CreationDate', u'TEXT', 0, None, 0),
 (5, u'Score', u'INTEGER', 0, None, 0),
 (6, u'ViewCount', u'INTEGER', 0, None, 0),
 (7, u'Body', u'TEXT', 0, None, 0),
 (8, u'OwnerUserId', u'INTEGER', 0, None, 0),
 (9, u'LastEditorUserId', u'INTEGER', 0, None, 0),
 (10, u'LastEditorDisplayName', u'TEXT', 0, None, 0),
 (11, u'LastEditDate', u'TEXT', 0, None, 0),
 (12, u'LastActivityDate', u'TEXT', 0, None, 0),
 (13, u'CommunityOwnedDate', u'TEXT', 0, None, 0),
 (14, u'ClosedDate', u'TEXT', 0, None, 0),
 (15, u'Title', u'TEXT', 0, None, 0),
 (16, u'Tags', u'TEXT', 0, None, 0),
 (17, u'AnswerCount', u'INTEGER', 0, None, 0),
 (18, u'CommentCount', u'INTEGER', 0, None, 0),
 (19, u'FavoriteCount', u'INTEGER', 0, None, 0)]